# Geometry Tutorial

This is an introductory walkthrough of the symbolic [geometry package](../api/symforce.geo.html) in symforce. Symforce natively contains the following geometry objects: complex numbers, quaternions, dual quaternions, matrices, 2D and 3D rotations, and 2D and 3D poses (translation + rotation).

In this tutorial we will demonstrate a few of the ways these geometry objects can be constructed and composed. Here we pay special attention to 3D rotations, poses, and vectors, as they are typically the most commonly used geometric objects.

In [1]:
# Setup
import symforce

symforce.set_backend("sympy")
symforce.set_log_level("warning")

from symforce.notebook_util import display
from symforce import sympy as sm
from symforce import geo
from symforce import ops

## Rotations

Rotations can be defined from and converted to a number of different representations as shown below, but always use a quaternion as the underlying representation. We use the notation world_R_body to represent a rotation that rotates a point written in the body frame into the world frame.

Note that in some cases a small epsilon can be provided to prevent numerical errors (e.g. division by zero) when converting to/from rotation representations. Furthermore, converting between certain representations can require additional symbolic expressions to guard against degenerate cases. For example, a rotation constructed from a rotation matrix results in more complexity than when constructed using an axis-angle representation as shown below.

In [2]:
# Identity definition
display(geo.Rot3())

<Rot3 <Q xyzw=[0, 0, 0, 1]>>

In [3]:
# Symbolic definition
display(geo.Rot3.symbolic("R"))

<Rot3 <Q xyzw=[R_x, R_y, R_z, R_w]>>

In [4]:
# To/From rotation matrix

# Rotate about x-axis
theta = sm.Symbol("theta")
R_mat = geo.Matrix(
    [[1, 0, 0], [0, sm.cos(theta), -sm.sin(theta)], [0, sm.sin(theta), sm.cos(theta)]]
)
R = geo.Rot3.from_rotation_matrix(R_mat)

display(R_mat)
display(R)  # Note the additional expressions required to avoid numerical errors
display(R.to_rotation_matrix())

⎡1    0        0   ⎤
⎢                  ⎥
⎢0  cos(θ)  -sin(θ)⎥
⎢                  ⎥
⎣0  sin(θ)  cos(θ) ⎦

<Rot3 <Q xyzw=[(1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1)))*(sqrt(Max(0, 2 - 2*cos(theta)))/2 + Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1))/2) + 2*sin(theta)*Min(1 - Max(0, sign(-2*cos(theta) + Max(1, 2*cos(theta) + 1, cos(theta)) - 1)), 1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1)), Min(1 - Max(0, sign(-cos(theta) + Max(1, 2*cos(theta) + 1, cos(theta)))), Max(1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1))), 1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1)), Min(1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1))), 1 - Max(0, sign(-cos(theta) + Max(1, 2*cos(theta) + 1, cos(theta))))))))))/(2*sqrt(Max(0, 2*cos(theta) + 2)) - 2*Min(1 - Max(0, sign(-2*cos(theta) + Max(1, 2*cos(theta) + 1, cos(theta)) - 1)), 1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1)), Min(1 - Max(0, sign(-cos(theta) + Max(1, 2*cos(theta) + 1, cos(theta)))), Max(1 - Max(0, 1 - Max(0, sig

⎡                                                                                                   
⎢                                                                                                   
⎢                                                                                                   
⎢                                                                                                   
⎢                                                                                                   
⎢                                                                                                   
⎢                                                                                                   
⎢                                                                                                   
⎢ ⎛1   Min(1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1))), 1 - Max(0, sign(-
⎢ ⎜─ - ────────────────────────────────────────────────────────────────────────────────────

In [5]:
# To/From Euler angles
R = geo.Rot3.from_yaw_pitch_roll(0, 0, theta)  # Yaw rotation only
ypr = R.to_yaw_pitch_roll()

display(R)
display(ops.StorageOps.simplify(list(ypr)))  # Simplify YPR expression

<Rot3 <Q xyzw=[sin(theta/2), 0, 0, cos(theta/2)]>>

In [6]:
# From axis-angle representation

# Rotate about x-axis
R = geo.Rot3.from_angle_axis(angle=theta, axis=geo.Vector3(1, 0, 0))

display(R)

<Rot3 <Q xyzw=[sin(theta/2), 0, 0, cos(theta/2)]>>

Now that we can construct rotations, we can use them to rotate vectors as one would expect.

In [7]:
world_R_body = geo.Rot3.symbolic("R")  # Rotation defining orientation of body frame wrt world frame
body_t_point = geo.Vector3.symbolic("p")  # Point written in body frame
world_t_point = world_R_body * body_t_point  # Point written in world frame
display(world_t_point)

⎡   ⎛       2        2    ⎞                                                         ⎤
⎢p₀⋅⎝- 2⋅R_y  - 2⋅R_z  + 1⎠ + p₁⋅(-2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + p₂⋅(2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z)⎥
⎢                                                                                   ⎥
⎢                               ⎛      2        2    ⎞                              ⎥
⎢p₀⋅(2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + p₁⋅⎝- 2⋅Rₓ  - 2⋅R_z  + 1⎠ + p₂⋅(-2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z) ⎥
⎢                                                                                   ⎥
⎢                                                            ⎛      2        2    ⎞ ⎥
⎣p₀⋅(-2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z) + p₁⋅(2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z) + p₂⋅⎝- 2⋅Rₓ  - 2⋅R_y  + 1⎠ ⎦

Chaining rotations and inverting rotations works as one would expect as well:

In [8]:
body_R_cam = geo.Rot3.symbolic("R_cam")
world_R_cam = world_R_body * body_R_cam

# Rotation inverse = negate vector part of quaternion
cam_R_body = body_R_cam.inverse()
display(body_R_cam)
display(cam_R_body)

<Rot3 <Q xyzw=[R_cam_x, R_cam_y, R_cam_z, R_cam_w]>>

<Rot3 <Q xyzw=[-R_cam_x, -R_cam_y, -R_cam_z, R_cam_w]>>

We can also easily substitute numerical values into symbolic expressions using geo objects themselves. This makes it very convenient to substitute numeric values into large symbolic expressions constructed using many different geo objects.

In [9]:
world_R_body_numeric = geo.Rot3.from_yaw_pitch_roll(0.1, -2.3, 0.7)
display(world_t_point.subs(world_R_body, world_R_body_numeric))

⎡-0.662947416398295⋅p₀ - 0.554353314451006⋅p₁ - 0.503182994394693⋅p₂ ⎤
⎢                                                                    ⎥
⎢-0.0665166116342196⋅p₀ + 0.713061539471145⋅p₁ - 0.697938952419008⋅p₂⎥
⎢                                                                    ⎥
⎣ 0.74570521217672⋅p₀ - 0.429226797490819⋅p₁ - 0.509596009450867⋅p₂  ⎦

## Poses

Poses are defined as a rotation plus a translation, and are constructed as such. We use the notation world_T_body to represent a pose that transforms from the body frame to the world frame.

In [10]:
# Symbolic construction
world_T_body = geo.Pose3.symbolic("T")
display(world_T_body)

<Pose3 R=<Rot3 <Q xyzw=[T.R_x, T.R_y, T.R_z, T.R_w]>>, t=(T.t0, T.t1, T.t2)>

In [11]:
# Construction from a rotation and translation

# Orientation of body frame wrt world frame
world_R_body = geo.Rot3.symbolic("R")

# Position of body frame wrt world frame written in the world frame
world_t_body = geo.Vector3.symbolic("t")

world_T_body = geo.Pose3(R=world_R_body, t=world_t_body)
display(world_T_body)

<Pose3 R=<Rot3 <Q xyzw=[R_x, R_y, R_z, R_w]>>, t=(t0, t1, t2)>

Similar to rotations, we can compose poses with poses, compose poses with 3D points, and invert poses as one would expect.

In [12]:
# Compose pose with a pose
body_T_cam = geo.Pose3.symbolic("T_cam")
world_T_cam = world_T_body * body_T_cam

# Compose pose with a point
body_t_point = geo.Vector3.symbolic("p")  # Position relative to body frame written in body frame
display(world_T_body * body_t_point)  # Equivalent to: world_R_body * body_t_point + world_t_body

⎡   ⎛       2        2    ⎞                                                              ⎤
⎢p₀⋅⎝- 2⋅R_y  - 2⋅R_z  + 1⎠ + p₁⋅(-2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + p₂⋅(2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z) + t₀⎥
⎢                                                                                        ⎥
⎢                               ⎛      2        2    ⎞                                   ⎥
⎢p₀⋅(2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + p₁⋅⎝- 2⋅Rₓ  - 2⋅R_z  + 1⎠ + p₂⋅(-2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z) + t₁ ⎥
⎢                                                                                        ⎥
⎢                                                            ⎛      2        2    ⎞      ⎥
⎣p₀⋅(-2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z) + p₁⋅(2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z) + p₂⋅⎝- 2⋅Rₓ  - 2⋅R_y  + 1⎠ + t₂ ⎦

In [13]:
# Invert a pose
body_T_world = world_T_body.inverse()
display(world_T_body)
display(body_T_world)

<Pose3 R=<Rot3 <Q xyzw=[R_x, R_y, R_z, R_w]>>, t=(t0, t1, t2)>

<Pose3 R=<Rot3 <Q xyzw=[-R_x, -R_y, -R_z, R_w]>>, t=(-t0*(-2*R_y**2 - 2*R_z**2 + 1) - t1*(2*R_w*R_z + 2*R_x*R_y) - t2*(-2*R_w*R_y + 2*R_x*R_z), -t0*(-2*R_w*R_z + 2*R_x*R_y) - t1*(-2*R_x**2 - 2*R_z**2 + 1) - t2*(2*R_w*R_x + 2*R_y*R_z), -t0*(2*R_w*R_y + 2*R_x*R_z) - t1*(-2*R_w*R_x + 2*R_y*R_z) - t2*(-2*R_x**2 - 2*R_y**2 + 1))>

## Vectors and matricies

Vectors and matrices are all represented using subclasses of geo.Matrix class, and can be constrcuted in several different ways as shown below.

In [14]:
# Matrix construction. The statements below all create the same 2x3 matrix object

# Construction from 2D list
m1 = geo.Matrix([[1, 2, 3], [4, 5, 6]])

# Construction using specified size + data
m2 = geo.Matrix(2, 3, [1, 2, 3, 4, 5, 6])

# geo.MatrixNM creates a matrix with shape NxM (defined by default 6x6 matrices and smaller)
m3 = geo.Matrix23(1, 2, 3, 4, 5, 6)
m4 = geo.Matrix23([1, 2, 3, 4, 5, 6])

# Construction using aliases
m5 = geo.M([[1, 2, 3], [4, 5, 6]])
m6 = geo.M(2, 3, [1, 2, 3, 4, 5, 6])
m7 = geo.M23(1, 2, 3, 4, 5, 6)
m8 = geo.M23([1, 2, 3, 4, 5, 6])

# Construction from block matrices of appropriate dimensions
m9 = geo.Matrix23.block_matrix([[geo.M13([1, 2, 3])], [geo.M13([3, 4, 5])]])

In [15]:
# Vector constructors. The statements below all create the same 3x1 vector object

# Construction from 2D list
v1 = geo.Matrix([[1], [2], [3]])

# Construction from 1D list. We assume a 1D list represents a column vector.
v2 = geo.Matrix([1, 2, 3])

# Construction using aliases (defined by default for 9x1 vectors and smaller)
v3 = geo.Matrix31(1, 2, 3)
v4 = geo.M31(1, 2, 3)
v5 = geo.Vector3(1, 2, 3)
v6 = geo.V3(1, 2, 3)

We can also use a few typical matrix constructors:

In [16]:
# Matrix of zeros
z1 = geo.Matrix23.zero()
z2 = geo.Matrix.zeros(2, 3)

# Matrix of ones
o1 = geo.Matrix23.one()
o2 = geo.Matrix.ones(2, 3)

Note that the Matrix class itself does not contain group or lie group methods, to prevent confusion
between the identity matrix and inverse matrix, and the group operations under addition.  The group
operations are implemented separately for matrices under addition, and are accessed through
ops.GroupOps and ops.LieGroupOps.

In [17]:
zero_matrix = geo.Matrix33.zero()  # We could also write ops.GroupOps.identity(geo.Matrix33)
identity_matrix = geo.Matrix33.eye()

display(zero_matrix)
display(identity_matrix)

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦

And, of course, matrix math works as one would expect:

In [18]:
# Matrix multiplication
m23 = geo.M23.symbolic("lhs")
m31 = geo.V3.symbolic("rhs")
display(m23 * m31)

⎡lhs₀ ₀⋅rhs₀ + lhs₀ ₁⋅rhs₁ + lhs₀ ₂⋅rhs₂⎤
⎢                                       ⎥
⎣lhs₁ ₀⋅rhs₀ + lhs₁ ₁⋅rhs₁ + lhs₁ ₂⋅rhs₂⎦

In [19]:
# Vector operations
norm = m31.norm()
squared_norm = m31.squared_norm()
unit_vec = m31.normalized()
display(unit_vec)

⎡           rhs₀           ⎤
⎢──────────────────────────⎥
⎢   _______________________⎥
⎢  ╱     2       2       2 ⎥
⎢╲╱  rhs₀  + rhs₁  + rhs₂  ⎥
⎢                          ⎥
⎢           rhs₁           ⎥
⎢──────────────────────────⎥
⎢   _______________________⎥
⎢  ╱     2       2       2 ⎥
⎢╲╱  rhs₀  + rhs₁  + rhs₂  ⎥
⎢                          ⎥
⎢           rhs₂           ⎥
⎢──────────────────────────⎥
⎢   _______________________⎥
⎢  ╱     2       2       2 ⎥
⎣╲╱  rhs₀  + rhs₁  + rhs₂  ⎦

In [20]:
m33 = 5 * geo.Matrix33.eye()  # Element-wise multiplication with scalar
display(m33.inv())  # Matrix inverse

⎡1/5   0    0 ⎤
⎢             ⎥
⎢ 0   1/5   0 ⎥
⎢             ⎥
⎣ 0    0   1/5⎦

One of the most powerful operations we can use matrices for is to compute jacobians with respect to other geo objects. By default we compute jacobians with respect to the tangent space of the given object.

In [21]:
R0 = geo.Rot3.symbolic("R0")
R1 = geo.Rot3.symbolic("R1")
residual = geo.M(R0.local_coordinates(R1))
display(residual)

⎡2⋅(2⋅Min(0, sign(R0_w*R1_w + R0_x*R1_x + R0_y*R1_y + R0_z*R1_z)) + 1)⋅(R_0_w⋅R₁ ₓ - R₀ ₓ⋅R_1_w - R_
⎢───────────────────────────────────────────────────────────────────────────────────────────────────
⎢                                                               ____________________________________
⎢                                                              ╱                                    
⎢                                                            ╲╱  1 - Min(1, Abs(R0_w*R1_w + R0_x*R1_
⎢                                                                                                   
⎢2⋅(2⋅Min(0, sign(R0_w*R1_w + R0_x*R1_x + R0_y*R1_y + R0_z*R1_z)) + 1)⋅(R_0_w⋅R_1_y + R₀ ₓ⋅R_1_z - R
⎢───────────────────────────────────────────────────────────────────────────────────────────────────
⎢                                                               ____________________________________
⎢                                                              ╱                           

In [22]:
jacobian = residual.jacobian(R1)
# The jacobian is quite a complex symbolic expression, so we don't display it for convenience
# The shape is equal to (dimension of residual) x (dimension of tangent space)
display(jacobian.shape)

## General properties of geo objects

### Storage operations

All geometric types implement the "Storage" interface. This means that they can:

1. Be serialized into a list of scalar expressions (`.to_storage()`)
2. Be reconstructed from a list of scalar expressions (`.from_storage()`)
3. Use common symbolic operations (symbolic construction, substitution, simplification, etc.)

In [23]:
# Serialization to scalar list
rot = geo.Rot3()
elements = rot.to_storage()
assert len(elements) == rot.storage_dim()
display(elements)

In [24]:
# Construction from scalar list
rot2 = geo.Rot3.from_storage(elements)
assert rot == rot2

In [25]:
# Symbolic operations
rot_sym = geo.Rot3.symbolic("rot_sym")
rot_num = rot_sym.subs(rot_sym, rot)

display(rot_sym)
display(rot_num)
display(rot_num.simplify())  # Simplify interal symbolic expressions
display(rot_num.evalf())  # Numerical evaluation

<Rot3 <Q xyzw=[rot_sym_x, rot_sym_y, rot_sym_z, rot_sym_w]>>

<Rot3 <Q xyzw=[0, 0, 0, 1]>>

<Rot3 <Q xyzw=[0, 0, 0, 1]>>

<Rot3 <Q xyzw=[0, 0, 0, 1.00000000000000]>>

### Group operations

All geometric types also implement the "Group" interface, meaning that geometric objects:

1. Can be composed with objects of the same type to produce an object of the same type (`.compose()`)
2. Have an identity element (`.identity()`)
3. Can be inverted (`.inverse()`)
4. Can be created to represent the relation between two other objects of the same type (`.between()`)

In [26]:
# Construct two random rotations
R1 = geo.Rot3.random()
R2 = geo.Rot3.random()

# Composition
display(R1.compose(R2))  # For rotations this is the same as R1 * R2

<Rot3 <Q xyzw=[-0.465329679341259, -0.803921440151188, -0.137709651676522, -0.343823587650020]>>

In [27]:
# Identity
R_identity = geo.Rot3.identity()
display(R1)
display(R_identity * R1)

<Rot3 <Q xyzw=[-0.0716641777881943, 0.755936526086563, -0.447138919565316, -0.472748348236229]>>

<Rot3 <Q xyzw=[-0.0716641777881943, 0.755936526086563, -0.447138919565316, -0.472748348236229]>>

In [28]:
# Inverse
R1_inv = R1.inverse()
display(R_identity)
display(R1_inv * R1)

<Rot3 <Q xyzw=[0, 0, 0, 1]>>

<Rot3 <Q xyzw=[0, 0, 0, 1.00000000000000]>>

In [29]:
# Between
R_delta = R1.between(R2)
display(R1 * R_delta)
display(R2)

<Rot3 <Q xyzw=[0.658908322442284, 0.441763180276511, -0.498006967437580, -0.350248733833403]>>

<Rot3 <Q xyzw=[0.658908322442284, 0.441763180276511, -0.498006967437580, -0.350248733833403]>>

### Lie Group operations

Rotations, poses, and matrices all implement the "LieGroup" interface, meaning that they each have a
tangent space. There are many great references on Lie groups out there already, so instead of
introducing them here, we recommend checking out
[Frank Dellaert's](https://github.com/borglab/gtsam/blob/develop/doc/LieGroups.pdf),
[Ethan Eade's](https://www.ethaneade.com/lie.pdf), or
[JL Blanco's](https://ingmec.ual.es/~jlblanco/papers/jlblanco2010geometry3D_techrep.pdf) tutorials.
In SymForce, objects which are a Lie Group can:

1. Be used to compute the tangent space vector about the identity element (`.to_tangent()`)
2. Be constructed from a tangent space vector about the identity element (`.from_tangent()`)
3. Be perturbed by a tangent space vector about the given element (`.retract()`)
4. Be used to compute the tangent space perturbation needed to obtain another given element (`.local_coordinates()`)
5. Be used to compute a jacobian describing the relation between the underlying data of the object (e.g. a quaternion for a rotation) and the tangent space vector about the given element (`.storage_D_tangent()`)

In [30]:
# To/From tangent space vector about identity element
R1 = geo.Rot3.random()
tangent_vec = R1.to_tangent()
R1_recovered = geo.Rot3.from_tangent(tangent_vec)

assert len(tangent_vec) == R1.tangent_dim()
display(R1)
display(R1_recovered)

<Rot3 <Q xyzw=[0.295521575429953, -0.432056001168596, -0.639177653433215, 0.563423941327673]>>

<Rot3 <Q xyzw=[0.295521575429953, -0.432056001168596, -0.639177653433215, 0.563423941327673]>>

In [31]:
# Tangent space perturbations

# Perturb R1 by the given vector in the tangent space around R1
R2 = R1.retract([0.1, 2.3, -0.5])

# Compute the tangent vector pointing from R1 to R2, in the tangent space around R1
recovered_tangent_vec = R1.local_coordinates(R2)

display(recovered_tangent_vec)

In [32]:
# Jacobian of storage w.r.t tangent space perturbation

# We chain storage_D_tangent together with jacobians of larger symbolic expressions taken
# with respect to the symbolic elements of the object (e.g. a quaternion for rotations) to compute
# the jacobian wrt the tanget space about the element.
# I.e. residual_D_tangent = residual_D_storage * storage_D_tangent

jacobian = R1.storage_D_tangent()
assert jacobian.shape == (R1.storage_dim(), R1.tangent_dim())

For more details on Storage/Group/LieGroup operations, see the [Concept tutorial](../notebooks/ops_tutorial.html).

## Poses and SE(n)

Poses in robotics are often represented using the [Special Euclidean group](https://en.wikipedia.org/wiki/Euclidean_group),
SE(2) and SE(3) for 2D and 3D respectively. Additionally, the manifold is often defined so that the
`from_tangent` function is the matrix exponential of a linear combination of generators.

This has the advantage of simpler theoretical treatment in some cases, but has disadvantages in
terms of the number of operations required to implement commonly used functions like `retract`.
For these reasons, instead of using the SE(n) `from_tangent` and `to_tangent` functions, ours
decouple the perturbations to rotation and translation.  This means our `Pose2` and `Pose3` objects
represent neither `SE(n)` nor `SO(n) x R^n`; they compose like objects on `SE(n)` but have tangent
space operations analogous to `SO(n) x R^n`.

For most users, this should all happen under the hood.  The most common observable difference is
that `retract(g, v) != compose(g, from_tangent(v))`, so simplifications done by hand that rely on
this fact must be treated with care.  We do also provide a `Pose3_SE3` class for users who need this
functionality.